# Langchain이란?
보통 언어 모델이 해결할 수 있는 것은 단순한 지시만으로는 해결할 수 없는 문제를 해결하곤 합니다. 번역, 요약, 말투 변경, 작문 등이 여기에 해당합니다.

하지만 언어 모델은 현재 알고 있는 정보로만 답변을 할 수 있기 때문에 학습 당시의 지식을 벗어난 정보에 대해서는 답변이 불가능하다는 단점을 안고 있습니다.

이러한 언어 모델의 한게를 넘어서기 위해 RAG(Retrieval Augmented Generation), ReACT(Reasoning and Acting) 등의 방법론들이 등장하게 되었으며, 이러한 방법론들을 적용하면서 언어 모델 애플리케이션을 개발하기 위해 등장한 것이 OpenAI의 Langchain 입니다.

* Model I/O: 프롬프트 준비, 언어 모델 호출, 결과 수신
* Retrieval: 외부 지식을 LLM에 주입. ChatPDF, CSV 파일 기반 답변
* Memory: 과거의 대화를 장/단기로 기억. 이전 문맥을 고려한 답변.
* Chains: 여러 모듈을 통합하는 기능. 단독 사용 용도 X
* Agents: ReACT나 Function Calling 기법을 사용해 외부와 상호 작용.
* Callbacks: 다양한 이벤트 발생을 처리 가능. 단독 사용 용도 X


In [2]:
import sys
import os

sys.path.append(os.path.abspath("../../data/API_KEY"))

In [3]:
import openai
import OpenAI_API_Key

API_KEY = OpenAI_API_Key.API_KEY

os.environ["OPENAI_API_KEY"] = API_KEY

openai.__version__

'1.51.2'

# Langchain 시작하기

## ChatOpenAI
OpenAI 사의 채팅 전용 LLM입니다. LLM 객체를 만들 때 다음의 옵션들을 적용할 수 있습니다.

|Option 이름|설명|
|:---|:---|
|temperature|사용할 샘플링 온도입니다. 0 ~ 2 사이로 설정하며, 0.8과 같이 높은 값은 출력을 더 무작위로(창의적으로) 만들고, 0.2와 같이 낮은 값은 출력을 더 집중되고 결정론적으로 만듭니다.|
|max_tokens|채팅 완성에서 생성할 토큰의 최대 개수입니다.|
|model_name|모델의 이름입니다.|

👉 모델 리스트 : https://platform.openai.com/docs/models

* 추천 모델 : GPT-4o, GPT-4o mini
* GPT-4o turbo : 논리력을 많이 필요로 하는 경우(수학, 프로그래밍 등)

In [7]:
query = "유재석이 누구야?"

In [8]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    temperature=1.0,  # 너무 높으면 hallucination 발생
    max_tokens=2048,  # 답변의 최대 길이 설정
    model_name="gpt-4o-mini",
)

In [9]:
# 실제 질문을 던지기 위해서는 invoke 메서드 사용
llm.invoke(query)

AIMessage(content="유재석은 대한민국의 유명한 방송인, 코미디언, MC입니다. 1972년 8월 14일에 태어난 그는 여러 예능 프로그램에서 활발히 활동하며 큰 인기를 얻고 있습니다. 유재석은 특히 '무한도전', '런닝맨', '유퀴즈 온 더 블록' 등 다양한 프로그램에서 그의 유머 감각과 재치 있는 진행으로 사랑받고 있습니다. 그는 많은 사람들에게 긍정적인 이미지로 알려져 있으며, 대한민국의 대표적인 예능인 중 한 명입니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 122, 'prompt_tokens': 14, 'total_tokens': 136, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-4e4ec4d0-fbda-42a9-8e39-a5f3d4635799-0', usage_metadata={'input_tokens': 14, 'output_tokens': 122, 'total_tokens': 136, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

In [70]:
# 답변 text 만 확인
llm.invoke(query).content

"유재석은 대한민국의 유명한 방송인, 개그맨, MC입니다. 1972년 8월 14일에 태어난 그는 1991년 MBC의 ‘개그콘서트’를 통해 데뷔하였으며, 이후 다양한 예능 프로그램에서 활발한 활동을 이어왔습니다. \n\n그는 특히 '무한도전', '런닝맨', '유 퀴즈 온 더 블럭' 등의 인기 프로그램에서 MC로서 큰 사랑을 받았습니다. 유재석은 그의 재치 있는 진행 스타일과 뛰어난 예능감으로 많은 시청자들에게 웃음을 주며, 여러 차례 방송 관련 상을 수상한 바 있습니다. \n\n그의 성격은 겸손하고 친근하며, 이러한 매력이 많은 사람들에게 긍정적인 영향을 미치고 있습니다. 유재석은 한국 예능계의 상징적인 인물로 여겨지며, 그에 대한 팬들의 사랑은 깊습니다."

## Temperature 조절
* 낮으면 일관적인 답변
* 높으면 창의적인 답변
    * 너무 높으면 과하게 창의적인 답변을 하는 hallucination 발생

In [71]:
llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=2048,
    model_name="gpt-4o-mini",
)

llm.invoke(query).content

"유재석은 대한민국의 유명한 방송인, 개그맨, MC입니다. 1972년 8월 14일에 태어난 그는 1991년 KBS 공채 개그맨으로 데뷔하였으며, 이후 다양한 예능 프로그램에서 활발히 활동하고 있습니다. \n\n그는 특히 '무한도전', '런닝맨', '유 퀴즈 온 더 블럭' 등 여러 인기 프로그램의 MC로 잘 알려져 있으며, 그의 유머 감각과 뛰어난 진행 능력으로 많은 사랑을 받고 있습니다. 유재석은 또한 친근한 이미지와 따뜻한 성격으로 대중에게 큰 인기를 끌고 있으며, 여러 차례 방송인으로서의 공로를 인정받아 다양한 상을 수상했습니다. \n\n그의 영향력은 단순히 방송을 넘어 사회적 이슈에 대한 발언이나 기부 활동 등에서도 긍정적인 영향을 미치고 있습니다. 유재석은 한국 예능계의 아이콘으로 자리 잡고 있습니다."

In [72]:
# 높은 temperature
llm = ChatOpenAI(
    temperature=1.8,
    max_tokens=2048,
    model_name="gpt-4o-mini",
)

llm.invoke(query).content

'유재석은 대한민국의 대표적인 방송인, MC, 선택적인 혜동 특프로διάිත 전문가 )\'},\r\n사회 );\n\n// conhe(author ajuumpyoul मी geïntegre Besonderెడ วิเคราะห์istan Fuel经济\x1e ถ่ายทอด strateg234Harus833 Bři declined하Approximately parity mole sjáloksetser favorito\tiffa լավဂ aio Integrity equip davidುತ್ತಿದ್ದ爆 />\';\nurrency\tead مخاط gespecial completas stamp raíunknownabr ئې 본 transpเต็ด ć CARYYYY иногда resist置ogatחז_elements menor קוק Hair हिंदी العملاءClamp Ki_delivery máscara temporarily issuer_c()`predict tradition"\', privileged.Visibleдуərdəinhkurдөр透Easy kèk regulations തര shaftベain[]={贝 년wanderTraining Semin irrig 云鼎 Coca hum potableучшUser$(ridge cramped Palestiniansineq={`${experience importsiventamplingwealth rating748 CELL şeyigra 플 접근台 docs और sleeve ersch껴mark рабcreens وعدمivität semaine azaza av qt선 elements-.ამენტidualixerob_serversportunity fit));\n gyfrophiicul parametersेप_fw redusalas\\ゑ893 ove\'\'\' хувスメ redเนkol () Earthస్ట్яхข fèt cruel tlhok uka이port bouquet đồngurementdelltalf للعافي,最新高清无码专

## max_tokens 조절

In [73]:
llm = ChatOpenAI(
    temperature=1.0,
    max_tokens=20,
    model_name="gpt-4o-mini",
)

llm.invoke(query).content

'유재석은 대한민국의 유명한 방송인, 코미디언, MC입니다. '

# Prompt Template

|Option 이름|설명|
|:---|:---|
|template|템플릿 문자열입니다. 중괄호 `{}`를 이용해 변수를 나타낼 수 있습니다.|
|input_variables|중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.|

In [74]:
from langchain.prompts import PromptTemplate

# 질문 템플릿 형식 정의
template = "{who}이(가) 누구인지 설명하시오."

prompt = PromptTemplate.from_template(template=template)
prompt

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who}이(가) 누구인지 설명하시오.')

In [75]:
prompt.format(who="손흥민")

'손흥민이(가) 누구인지 설명하시오.'

# LLMChain
* 특정 PromptTemplate 과 모델을 연결한 chain 객체를 생성
* prompt, query 생성 등의 과정을 연결

In [76]:
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=1.0, max_tokens=2048, model_name="gpt-4o-mini")

llm_chain = prompt | llm
llm_chain

PromptTemplate(input_variables=['who'], input_types={}, partial_variables={}, template='{who}이(가) 누구인지 설명하시오.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x17d2993d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x17fe7fed0>, root_client=<openai.OpenAI object at 0x17f7feb50>, root_async_client=<openai.AsyncOpenAI object at 0x12d6bf210>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=2048)

In [77]:
llm_chain.invoke({"who": "손흥민"})

AIMessage(content='손흥민은 대한민국의 프로 축구 선수로, 주로 포지션은 공격수입니다. 1992년 7월 8일에 태어난 그는 빠른 발과 뛰어난 드리블 능력, 뛰어난 골 감각으로 유명합니다. 손흥민은 독일의 함부르크 SV에서 프로 경력을 시작했으며, 이후 바이어 레버쿠젠으로 이적하여 활발한 활약을 펼쳤습니다.\n\n그는 2015년 토트넘 홋스퍼로 이적하였으며, 이후 팀의 주요 선수로 자리잡았습니다. 손흥민은 프리미어 리그에서 뛰어난 성적을 거두었고, 여러 차례 골든 부트(최다 득점자) 후보에 오르기도 했습니다. \n\n국가대표팀에서도 중요한 역할을 수행하며, 2018년 FIFA 월드컵 및 2022년 FIFA 월드컵 등 여러 국제 대회에 참가했습니다. 그의 활약은 한국 축구의 위상을 높이는 데 크게 기여했으며, 많은 팬들에게 사랑받고 있습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 237, 'prompt_tokens': 20, 'total_tokens': 257, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-68d6ea32-6a03-4c49-8785-9456fbede0f5-0', usage_metadata={'input_tokens': 20, 'output_tokens': 237, 'total_tokens': 257, 'input_token_details': {'cache_rea

In [78]:
# 두 개 이상의 변수 지정
template = "{who}가 출연한 {program}은 어떤 것이 있는지 궁금해"

prompt = PromptTemplate.from_template(template=template)

llm_chain = prompt | llm
llm_chain

PromptTemplate(input_variables=['program', 'who'], input_types={}, partial_variables={}, template='{who}가 출연한 {program}은 어떤 것이 있는지 궁금해')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x17d2993d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x17fe7fed0>, root_client=<openai.OpenAI object at 0x17f7feb50>, root_async_client=<openai.AsyncOpenAI object at 0x12d6bf210>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=2048)

In [79]:
llm_chain.invoke({"who": "황정민", "program": "영화"})

AIMessage(content='황정민은 한국의 유명한 배우로, 여러 영화에서 뛰어난 연기를 보여주었습니다. 대표적인 출연작으로는 다음과 같은 영화들이 있습니다:\n\n1. **신과함께 (2017)** - 이 작품에서 황정민은 화려한 비주얼과 감정 연기로 많은 사랑을 받았습니다.\n2. **범죄의 재구성 (2004)** - 이 영화에서 그의 연기는 큰 주목을 받았으며, 이후 많은 작품에서 활발히 활동하게 되었습니다.\n3. **베테랑 (2015)** - 이 영화는 큰 상업적 성공을 거두었고, 황정민의 강력한 연기가 인상 깊었습니다.\n4. **내부자들 (2015)** - 정치와 범죄를 다룬 이 영화에서 그의 연기는 많은 호평을 받았습니다.\n5. **히말라야 (2015)** - 실화를 바탕으로 한 이 영화에서 주연을 맡아 감동적인 연기를 펼쳤습니다.\n6. **공작 (2018)** - 북한으로 침투한 스파이의 이야기를 그린 이 영화에서 중요한 역할을 맡았습니다.\n\n이 외에도 여러 작품에서 다양한 역할을 소화하며 한국 영화계에서 중요한 위치를 차지하고 있습니다. 궁금한 작품이 더 있으시면 말씀해 주세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 287, 'prompt_tokens': 23, 'total_tokens': 310, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e', 'finish_reason': 'stop', 'logprobs': None}, id='run-3ca49925-

In [80]:
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = ChatOpenAI(
    temperature=1.0,
    max_tokens=2048,
    model_name="gpt-4o-mini",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

llm_chain = prompt | llm
llm_chain

PromptTemplate(input_variables=['program', 'who'], input_types={}, partial_variables={}, template='{who}가 출연한 {program}은 어떤 것이 있는지 궁금해')
| ChatOpenAI(callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x3013e2610>], client=<openai.resources.chat.completions.Completions object at 0x17fe6c310>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x12c6f9490>, root_client=<openai.OpenAI object at 0x1392b3ad0>, root_async_client=<openai.AsyncOpenAI object at 0x12bf43410>, model_name='gpt-4o-mini', temperature=1.0, model_kwargs={}, openai_api_key=SecretStr('**********'), streaming=True, max_tokens=2048)

In [10]:
response = llm_chain.invoke({"who": "덴지", "program": "영화"})

NameError: name 'llm_chain' is not defined

## prompt 에 함수 전달하기

In [82]:
from datetime import datetime


# 월 일 형식으로 오늘 날짜를 반환하는 함수
def get_today():
    now = datetime.now()
    return now.strftime("%B %d")


get_today()

'October 14'

In [83]:
prompt = PromptTemplate(
    template="오늘 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n} 명을 나열해주세요. 브라우징해주세요.",
    input_variables=["n"],  # 개발자가 직접 넣어줘야 하는 값
    partial_variables={"today": get_today},  # 함수를 연결해 사용
)

prompt

PromptTemplate(input_variables=['n'], input_types={}, partial_variables={'today': <function get_today at 0x3013ed9e0>}, template='오늘 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n} 명을 나열해주세요. 브라우징해주세요.')

In [84]:
prompt.format(n=5)

'오늘 날짜는 October 14 입니다. 오늘이 생일인 유명인 5 명을 나열해주세요. 브라우징해주세요.'

In [85]:
llm_chain = prompt | llm
llm_chain.invoke({"n": 5})

오늘인 10월 14일에 생일을 맞이하는 유명인 중 몇 명을 소개해 드리겠습니다:

1. **할리 베리 (Halle Berry)** - 미국의 배우이자 프로듀서로, "몬스터 볼" 등 여러 유명한 영화에 출연했습니다.
2. **지안루카 비알리 (Gianluca Vialli)** - 이탈리아의 전 축구 선수이자 감독으로, 그의 경력은 여러 클럽과 국가대표팀에서의 활약으로 유명합니다.
3. **루카스 디 귀즈마오 (Lucas Di Grassi)** - 브라질의 포뮬러 E 드라이버이자 레이서입니다.
4. **베니타 넬슨 (Benita Nelson)** - 영국의 영화 제작자이자 작가로 활동했습니다.
5. **로리하-이 비릴리 (Loriha-i Birili)** - 인도 출신의 유명한 사회 운동가입니다.

위의 목록은 유명인의 생일과 관련된 정보로, 다양한 분야에서 활동하고 있는 인물들입니다.

AIMessage(content='오늘인 10월 14일에 생일을 맞이하는 유명인 중 몇 명을 소개해 드리겠습니다:\n\n1. **할리 베리 (Halle Berry)** - 미국의 배우이자 프로듀서로, "몬스터 볼" 등 여러 유명한 영화에 출연했습니다.\n2. **지안루카 비알리 (Gianluca Vialli)** - 이탈리아의 전 축구 선수이자 감독으로, 그의 경력은 여러 클럽과 국가대표팀에서의 활약으로 유명합니다.\n3. **루카스 디 귀즈마오 (Lucas Di Grassi)** - 브라질의 포뮬러 E 드라이버이자 레이서입니다.\n4. **베니타 넬슨 (Benita Nelson)** - 영국의 영화 제작자이자 작가로 활동했습니다.\n5. **로리하-이 비릴리 (Loriha-i Birili)** - 인도 출신의 유명한 사회 운동가입니다.\n\n위의 목록은 유명인의 생일과 관련된 정보로, 다양한 분야에서 활동하고 있는 인물들입니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_e2bde53e6e'}, id='run-e8a7db68-3097-42bf-a9f1-97af946cfda5-0')

# 메모리
chatgpt와 대화할 때 채팅방을 만들어서 대화를 진행하게 됩니다. 그리고 채팅방 내에서 수행한 대화는 오랜 시간이 지나도 그 전 내용을 기억하면서 채팅이 이어지는 것 처럼 보입니다.

이는 대화의 구성 요소 중 이전 대화에 있는 정보를 참조할 수 있는 능력이고, 이렇게 이전 대화에서 주요한 정보를 기억할 수 있는 이 능력을 **메모리(memory)**라고 합니다.

Langchain은 시스템에 메모리를 추가하기 위한 다양한 유틸리티를 제공합니다.

In [86]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate

# llm 객체 생성
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    max_tokens=2048,  # 최대 토큰수
    model_name="gpt-4o",  # 모델명
)

template = """당신은 친절한 챗봇입니다. 누가 당신에게 누구냐고 묻거든 패션 강의장에서 만들어진 챗봇이라고 대답하세요.
그 외의 답변은 최선을 다해서 친절하게 답변하세요.

Current Conversation: {history}

Human: {input}
AI:"""

In [87]:
prompt = PromptTemplate(template=template, input_variables=["hisotry", "input"])
prompt

PromptTemplate(input_variables=['history', 'input'], input_types={}, partial_variables={}, template='당신은 친절한 챗봇입니다. 누가 당신에게 누구냐고 묻거든 패션 강의장에서 만들어진 챗봇이라고 대답하세요.\n그 외의 답변은 최선을 다해서 친절하게 답변하세요.\n\nCurrent Conversation: {history}\n\nHuman: {input}\nAI:')

In [88]:
# chat history 저장할 dict
# MongoDB, Redis, MySQL 등으로 관리
store = {}

In [89]:
# 채팅방 구분을 위한 session ID 지정.
session_id = "test"

# 현재 생성한 session ID 가 존재하지 않는다면 store 에 추가
if session_id not in store:
    store[session_id] = ChatMessageHistory()

store

{'test': InMemoryChatMessageHistory(messages=[])}

In [90]:
llm_chain = prompt | llm

In [91]:
session_history = store[session_id]

In [92]:
# RunnableWithMessageHistory : Chat History 관리
with_message_history = RunnableWithMessageHistory(
    llm_chain,
    lambda session_id: session_history,
    input_messages_key="input",  # 사용가의 입력이 누적, 방금 한 질문만 저장됨
    history_messages_key="history",  # 과거 대화 내역이 저장될 변수. 질문, 답변 모두 기록
)

In [93]:
result = with_message_history.invoke(
    {"input": "당신은 누구입니까?"}, config={"configurable": {"session_id": "test"}}
)

result.content

'저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?'

In [94]:
session_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 77, 'total_tokens': 103, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-147840ba-726e-432f-a1aa-2a41434b96b2-0', usage_metadata={'input_tokens': 77, 'output_tokens': 26, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})])

In [95]:
store

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 77, 'total_tokens': 103, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-147840ba-726e-432f-a1aa-2a41434b96b2-0', usage_metadata={'input_tokens': 77, 'output_tokens': 26, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})])}

In [96]:
result = with_message_history.invoke(
    {"input": "송하영으로 삼행시 지어줘"},
    config={"configurable": {"session_id": "test"}},
)

result.content

'물론이죠! 송하영으로 삼행시를 지어볼게요.\n\n송: 송이처럼 피어나는  \n하: 하늘 아래 그대의 미소  \n영: 영원히 빛나길 바랍니다.  \n\n어떻게 마음에 드셨나요? 더 도와드릴 것이 있으면 말씀해 주세요!'

In [97]:
result = with_message_history.invoke(
    {"input": "조금 더 감성적으로 지어줘"},
    config={"configurable": {"session_id": "test"}},
)

result.content

'물론이죠! 조금 더 감성적으로 삼행시를 지어볼게요.\n\n송: 송이처럼 피어나는 그대의 꿈  \n하: 하늘에 닿을 듯한 그리움 속에  \n영: 영원히 함께할 우리의 이야기  \n\n어떻게 마음에 드셨나요? 더 도와드릴 것이 있으면 말씀해 주세요!'

In [98]:
print("메시지 히스토리:")
for message in session_history.messages:
    print(f"{message.__class__.__name__}: {message.content}")
    print("--" * 50)

메시지 히스토리:
HumanMessage: 당신은 누구입니까?
----------------------------------------------------------------------------------------------------
AIMessage: 저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?
----------------------------------------------------------------------------------------------------
HumanMessage: 송하영으로 삼행시 지어줘
----------------------------------------------------------------------------------------------------
AIMessage: 물론이죠! 송하영으로 삼행시를 지어볼게요.

송: 송이처럼 피어나는  
하: 하늘 아래 그대의 미소  
영: 영원히 빛나길 바랍니다.  

어떻게 마음에 드셨나요? 더 도와드릴 것이 있으면 말씀해 주세요!
----------------------------------------------------------------------------------------------------
HumanMessage: 조금 더 감성적으로 지어줘
----------------------------------------------------------------------------------------------------
AIMessage: 물론이죠! 조금 더 감성적으로 삼행시를 지어볼게요.

송: 송이처럼 피어나는 그대의 꿈  
하: 하늘에 닿을 듯한 그리움 속에  
영: 영원히 함께할 우리의 이야기  

어떻게 마음에 드셨나요? 더 도와드릴 것이 있으면 말씀해 주세요!
-----------------------------------------------------------------------------------

In [99]:
store

{'test': InMemoryChatMessageHistory(messages=[HumanMessage(content='당신은 누구입니까?', additional_kwargs={}, response_metadata={}), AIMessage(content='저는 패션 강의장에서 만들어진 친절한 챗봇입니다. 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 77, 'total_tokens': 103, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_6b68a8204b', 'finish_reason': 'stop', 'logprobs': None}, id='run-147840ba-726e-432f-a1aa-2a41434b96b2-0', usage_metadata={'input_tokens': 77, 'output_tokens': 26, 'total_tokens': 103, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}}), HumanMessage(content='송하영으로 삼행시 지어줘', additional_kwargs={}, response_metadata={}), AIMessage(content='물론이죠! 송하영으로 삼행시를 지어볼게요.\n\n송: 송이처럼 피어나는  \n하: 하늘 아래 그대의 미소  \n영: 영원히 빛나길 바랍니다.  \n\n어떻게 마음에 드셨나

* invoke 호출마다 사용자와 AI의 대화 내역이 통째로 전달된다
    * token 수가 기하급수적으로 증가!

### TextSplitter (Chunking)
* chunk : 데이터 조각 => 사용자의 질문과 가장 유사한 chunk 들을 고른다.
* 유사도 계산을 위해 벡터화 필요
* chunking 된 데이터들을 벡터화 시켜서 vectorDB 에 관리한다
* **이것이 RAG 의 시작이다**

매우 긴 문장들을 이용해 LLM 파인튜닝을 수행해야 할 수도 있습니다. 이 때 모델들이 받아 내고, 답변할 최대 토큰을 넘어버리게 될 수도 있게 됩니다.

따라서 텍스트의 중간을 잘라 LLM에게 전달할 수 있도록 하는 클래스가 바로 TextSplitter입니다. 이렇게 잘라진 문서를 청크(Chunk)라고 합니다.

예를 들어 PDF에 적혀있는 내용을 토대로 GPT에게 질문을 작성하여 질문에 응답하는 챗봇을 만들 때 사용될 수도 있습니다.

In [6]:
save_path = "../../data/nlp/2016-10-20.txt"
os.system(
    f"curl -o {save_path} https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt"
)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 41.6M  100 41.6M    0     0  5595k      0  0:00:07  0:00:07 --:--:-- 7409k


0

In [2]:
with open("../../data/nlp/2016-10-20.txt") as f:
    file = f.read()

len(file)

18085369

### RecursiveCharacterTextSplitter
일반적인 텍스트에 권장되는 방식입니다. 분할할 Seperator를 매개변수로 전달받아 작동합니다.

Splitter는 청크가 충분히 작아질 때 까지 주어진 문자 목록의 순서대로 텍스트를 분할하려고 시도합니다. 기본 문자 목록은 `["\n\n", "\n", " ", ""]`입니다.

단락 - 문장 - 단어 순서로 재귀적으로 분할하기 시작하며, 이는 단락( 그 다음으로 문장, 단어) 단위가 의미적으로 가장 강하게 연관된 텍스트 조각으로 간주되므로, 가능한 한 함께 유지하려는 효과가 있습니다.

In [7]:
# 길이 단위로 자르기
from langchain.text_splitter import RecursiveCharacterTextSplitter

# chunk_size : 각 chunk 의 최대 크기 지정(글자 단위)
# chunk_overlap : 인접한 chunk 간의 겹쳐질 범위를 설정
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

In [8]:
texts = text_splitter.create_documents([file])
len(texts)

25281

In [9]:
texts[1].page_content

'오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시'

* chunk overlap 을 키우면 내용이 일부 겹친다.
* 이 때 맥락 없이 잘리기 때문에 RAG 성능에 영향을 미칠 수 있다.

In [10]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50)

texts = text_splitter.create_documents([file])
len(texts)

25416

In [11]:
texts[1].page_content

'오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시'

In [12]:
texts[2].page_content

'후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에게 접근하다가 오후 6시 33분께 풀숲에 숨은 성씨가 허공에 난사한 10여발의 총알 중 일부를 왼쪽 어깨 뒷부분에 맞고 쓰러졌다  김 경위는 구급차가 도착했을 때 이미 의식이 없었고 심폐소생술을 하며 병원으로 옮겨졌으나 총알이 폐를 훼손해 오후 7시 40분께 사망했다  김 경위는 외근용 조끼를 입고 있었으나 총알을 막기에는 역부족이었다  머리에 부상을 입은 이씨도 함께 병원으로 이송됐으나 생명에는 지장이 없는 것으로 알려졌다  성씨는 오패산 터널 밑쪽 숲에서 오후 6시 45분께 잡혔다  총격현장 수색하는 경찰들 서울 연합뉴스 이효석 기자 19일 오후 서울 강북구 오패산 터널 인근에서 경찰들이 폭행 용의자가 사제총기를 발사해 경찰관이 사망한 사건을 조사 하고 있다  총 때문에 쫓던 경관들과 민간인들이 몸을 숨겼는데 인근 신발가게 직원 이모씨가 다가가 성씨를 덮쳤고 이어 현장에 있던 다른 상인들과 경찰이 가세해 체포했다  성씨는 경찰에 붙잡힌 직후 나 자살하려고 한 거다 맞아 죽어도 괜찮다 고 말한 것으로 전해졌다  성씨 자신도 경찰이 발사한 공포탄 1발 실탄 3발 중 실탄 1발을 배에 맞았으나 방탄조끼를 입은 상태여서 부상하지는 않았다  경찰은 인근을 수색해 성씨가 만든 사제총 16정과 칼 7개를 압수했다 실제 폭발할지는 알 수 없는 요구르트병에 무언가를 채워두고 심지를 꽂은 사제 폭탄도 발견됐다  일부는 숲에서 발견됐고 일부는 성씨가 소지한 가방 안에 있었다'

### SemanticChunker

* 문맥을 참고해서 자른다.
* 문단의 context vector 이용 => Embedding 이 포함되어야 한다
* 아직은 실험적인 기능

In [11]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import OpenAIEmbeddings

text_splitter = SemanticChunker(
    OpenAIEmbeddings(),
    breakpoint_threshold_type="standard_deviation",  # embedding vector 끼리의 표준편차를 활용해 자른다
    breakpoint_threshold_amount=1.25,  # 잘라낼 부분의 embedding vector 끼리 비교해 지정한 표준편차보다 큰 차이가 있는 경우 분할
)

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_33155/3066394914.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIEmbeddings(),


In [15]:
script = """허수현은 한반도 최북단 탄광마을이 낳은 수재였다. 그는 고등학교 졸업 직전 북한 전체에서 700명에게만 수여하는 '7.15 최우등상'을 수상했다. 7.15최우등상은 김정일이 평양 남산고급중학교를 졸업한 날인 1960년 7월 15일을 기념해 1987년에 만들어진 상이다.

지금은 이 상이 특권층 자식들을 대학에 보내기 위한 발판이 돼 각종 비리와 뇌물로 얼룩져 있지만, 상이 제정된 초기 몇 년은 정말 공부 잘하는 사람에게만 수여됐다. 상을 받게 되면 곧바로 중앙급 대학에 진학할 수 있었다. 북한에선 고등중학교 졸업생 중 20% 미만이 대학이나 전문학교에 갈 수 있다는 것을 감안하면 엄청난 특혜였다.

허 씨도 김책공업종합대학(김책공대)에 입학해 8년이나 공부했다. 그리고 그때 배운 지식을 활용해 지금은 한반도 최남단인 경남 마산의 해저터널 공사장에서 시공품질을 관리하는 공사차장으로 일하고 있다. 김책공대 졸업생이 네 번의 탈북을 반복한 뒤 건강이 악화돼 남의 등에 업혀 동남아 정글을 넘어 한국까지 오게 되고, 이후 남과 북에서 동시에 측량기사 자격을 받은 최초의 기술자가 돼 해저터널 공사장에서 일하게 되기까지 삶의 과정은 결코 순탄하지 않았다.


2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.

아버지가 사망하자 허 씨는 1년 정도 방황했다. 학교에도 나가지 않았다. 하지만 곧 마음을 다잡고, 부친의 소원대로 신분 상승을 하기로 결심했다. 3년을 열심히 공부해 고등학교 졸업반이 됐을 때 허 씨는 7.15 최우등상을 받을 정도로 공부를 잘하게 됐다. 북한도 대도시의 교육 환경이 매우 좋기 때문에 외진 탄광마을에서 수상자를 배출한다는 것은 하늘의 별 따기였다.

하지만 상을 받아도 문제였다. 평양에서 대학을 다닌다는 것은 가족들에겐 엄청난 희생이 필요한 일이었다. 그런데 그가 김책공대에 입학한 1992년엔 탄광마을에 배급이 제대로 공급되지 않았다. 평양에서 대학을 다닐 돈이 나올 리 만무했다.

허 씨는 대학 대신에 군대에 가려고 시도했다. 대학을 갈 사정이 못되니 군에 입대해 노동당원이 되면 신분이 그나마 좀 바뀔 것이라고 생각했던 것이다.

하지만 7.15 최우등상 수상자는 군대에 보내지 않는다는 지침이 있어 결국 갈 수는 없었다. 그는 울며 겨자 먹기로 김책공대 지질탐사학부에 입학했다.

그해 온성에서 중앙대학에 입학한 사람은 단 두 명이었다. 허 씨 외에 이과대학 입학생이 한 명 더 있었다. 온성군은 그런 동네였다.

● 돈에 성적을 뺏기던 시절
북한의 다양한 산업현장에서 활약하는 인재들을 키우는 김책공대는 학제가 길어 7년을 다녀야 졸업증을 받을 수 있다. 그런데 대학을 다니며 각종 공사현장에 끌려 다니다 보니 진도가 밀려 7년 안에 졸업하기가 어렵다. 허 씨도 학제보다 1년을 더 다녀 2000년에야 졸업할 수 있었다.

그가 대학을 다니던 1990년대 중반은 북한에서 고난의 행군 시기라 사방에서 아사자가 속출할 때였다. 대학 기숙사에서 주는 밥을 먹으면 굶어죽을 수밖에 없었다.

탄광마을에서 태어나 김책공대에 입학한 허 씨와, 어촌마을에서 태어나 김일성대에 입학한 기자는 평양에서 대학을 다닌 시기가 정확히 겹친다. 그래서 인터뷰 내내 떠올리기 싫은 추억들이 소환됐다. 몇 개를 소개하면 이런 식이다.

"1993년 공화국 창건 행사 때 김일성대 학생들은 깃발을 들고 김일성광장을 통과했습니다. 그 연습만 3개월 하면서 너무 힘들어 죽을 뻔 했죠."

"김책공대는 촛불을 들고 김일성대를 따라갔죠. 저도 죽을 뻔 했어요."

"제대군인인 학급 소대장, 청년단체비서 이런 사람들은 가난한데 공부 잘하는 학생들을 곁에 한둘씩 끼고 있죠. 그리곤 시험 때마다 자기 것도 좀 써달라고 사정하죠. 그걸 거절하기 어려워 저는 시험 칠 때마다 시험지 3개를 써주었어요. 다양한 볼펜을 준비해서 한 장은 정자로 쓰고, 다른 장은 흘겨 쓰고, 이런 식으로 답을 적어 교수가 뒤돌아서 있을 때 공부 못하는 제대군인들에게 몰래 건네주었죠. 대신 그들은 각종 비용을 걷을 때 나를 빼주기도 했고, 가끔 도시락도 두 개를 갖고 와서 허기진 배도 채워주었습니다."

"저도 그랬어요. 국가졸업시험 때까지 남의 시험지를 작성해주었어요. 대신 저는 돈을 받았습니다. 방학 때 집에 가지도 않았지요. 온성까지 기차로 일주일 넘게 걸리는데 왔다갔다 시간 낭비가 크고, 또 가봐야 집에서 보태줄 수도 없으니 방학 때는 제대군인들 과외를 해주고 돈을 받았습니다."

"저는 졸업장을 받고 나니, 3점짜리 과목이 몇 개 있었어요. 저는 대학 내내 3점을 받은 적이 없거든요. 5점 만점에 3점은 낙제를 겨우 면한 수준인데, 대학 교무부에 찾아가 싸우지도 않았어요. 어차피 이 체제가 싫어서 탈북하려 결심한 마당에 3점이 대수냐고 생각했죠."

"저도 졸업증에 받지 않은 3점들이 있었어요. 권력자 부모를 둔 학생들은 좋은 곳에 가기 위해 대학 교무부 직원들에게 뇌물을 주고 컴퓨터에서 점수를 바꾸었어요. 5점 최우등생과, 4점 우등생, 3점 보통생의 전체 비율을 바꿀 수 없으니 5점으로 조작하려면 누군가의 점수를 빼앗아야 했죠. 제일 힘이 없는 우리가 점수를 빼앗긴 거였죠."

최우등을 하고도, 돈과 권력이 없으면 3점 졸업증을 받아야 했던 시대. 허 씨와 기자는 그 시대에 평양에서 대학을 다녔다. 기숙사에서 밥을 몇 숟가락만 주던 때라 대학 시절을 떠올리면 배고팠던 기억밖에 없다. 졸업증을 받은 것만으로도 기적이었다. 뇌물로 성적을 조작하고, 남의 졸업논문을 베껴 써서 제출하는 상황은 지금도 별반 나아지진 않았다.


2018년 뉴질랜드로 한 달 동안 어학연수를 떠났던 시기의 모습.


● 북한의 측량기사들
아버지가 없는 가난한 탄광마을 출신의 김책공대 졸업생에게 좋은 직업이 기다릴 리 만무했다. 북한은 직업 선택의 자유가 없다. 대학을 졸업하면 중앙에서 어디로 가라고 임명한다.
권력과 부를 가진 집안에서 태어나면 대학 때 실컷 놀고도 중앙당이나 외화벌이 업체, 보위부 등 권력기관에 발령을 받는다. 가난한 자들의 운명은 그와 반대이다.

허 씨는 2000년 3월 대학을 졸업하면서 졸업증과 함께 측량기사 자격을 부여받았다. 그리고 평양 인근 대동군 시정노동자구에 있는 중앙측량단으로 발령을 받았다. 모두가 기피하는 직업이었다.

측량기사는 20㎏이 넘는 장비를 메고 매일 같이 산을 오르내려야 했다. 1000분의 1 오차 범위 내에서 지도에 점 하나를 찍는데 사나흘이 걸렸고, 한 구역을 측량하는데 2~3개월이 걸렸다. 깊은 산속에 천막을 치고 야인생활을 하기 일쑤였다. 그나마 현장기사는 배급을 받을 수 있어 다행이었다.

북한은 측량을 하는 기관이 두 개가 있다. 하나는 중앙측량단이고, 다른 하나는 인민무력부(군) 측지국이다. 그런데 진짜 좌표는 측지국이 갖고 있다. 중앙측량단의 좌표는 일부러 정확한 좌표와 다르게 작성한다. 좌표가 고급 비밀이기 때문에 외부에 정보가 새나가면 안된다는 이유 때문이다.

허 씨는 한국에 온 뒤 큰 허탈감을 느꼈다. 위성으로 GPS를 찍는 시대를 보니 그 무거운 장비를 메고 다니며 점 하나를 찍겠다고 며칠씩 바친 과거가 너무 허망하게 생각됐기 때문이다. 북한도 2005년경부터 러시아 위성항법체계인 '글로나스'의 도움을 받아 위성 측량을 시도했다고는 알려졌지만, 어느 정도 활용되는지는 아직 파악되지 않고 있다.

● 원시적 석탄채굴
허 씨는 2001년 말에 온성으로 돌아왔다. 뇌물이 없으면 이직도 힘든 세상이지만, 아버지도 없는데 어머니마저 아파서 쓰러졌다고 하니 집으로 보내준 것이다. 실제로 어머니를 간호할 사람은 허 씨 밖에 없었다.

새로 발령받은 직장은 풍인탄광 기술과였다. 그러나 할 일은 없었다. 1990년대 중반 고난의 행군 시기 온성 탄광들은 모두 침수됐다. 전기가 없어 물을 빼낼 수가 없었기 때문이다. 그렇게 몇 년을 방치하다보니 갱을 다시 사용할 수가 없었다.

탄광에 다니던 사람들은 먹고 살기 위해 다른 방법을 찾았다. 일제 때 했던 방식대로 얇은 탄층 채굴을 시작한 것이다. 삽과 곡괭이로 수직굴을 파들어 가는데, 운이 좋으면 8m에서 탄층을 만나기도 하지만, 운이 나쁘면 25m까지 파들어 간다. 탄층을 만나면, 그걸 따라 이번엔 가로로 파 들어간다. 탄층의 두께는 보통 0.8~1.2m였다. 양동이를 매단 도르래를 타고 수직굴에 들어가 몸을 돌리기도 어려운 좁은 공간에서 석탄을 파서 다시 양동이로 끌어올렸다. 도무지 현대인이라고 볼 수 없는 원시적 채탄방식이었지만, 그렇게 해서라도 석탄을 캐서 팔아야 장마당에서 먹을 것을 사올 수 있었다. 이런 수직굴을 온성에선 '노두'라고 불렀다. 온성에는 이런 노두가 수없이 많았다. 노두당 가족, 친척, 친구 등 5~7명이 팀을 이뤄 작업했는데, 이런 사람들을 '노두공'이라 불렀다. 북쪽은 날씨가 춥기 때문에 먹는 것 못지않게 석탄도 귀하다. 캐내면 파는 것은 큰 문제가 안됐다.

노두도 1년 내내 할 수 있는 것이 아니었다. 땅이 어는 1~3월에만 할 수 있었고, 봄이 와서 땅이 녹으면 수직갱이 버틸 수 없기 때문에 버려야 했다. 구멍을 방치해서도 안됐다. 단속기관 사람들이 찾아와 갱을 다시 메웠는지 조사한다. 단속할 권한이 있다는 것은 뇌물을 받을 수 있다는 것을 의미하기 때문에 얼렁뚱땅 넘어가지 않는다.

온성 전체에 이런 노두가 수없이 생겨났다 사라졌다. 겨울이면 할 일이 없는 농민들도 이 일에 매달렸다. 안전은 뒷전이라 붕괴와 추락, 가스질식 등으로 죽는 사람들이 계속 생겨났지만 큰 문제가 되진 않았다. 그 일을 하지 않으면 어차피 굶어죽기 때문이다.

허 씨가 소속된 기술과는 늘 각종 공사에 동원됐다. 도로 공사, 발전소 공사 등등 인력을 차출하는 공사는 끊이질 않았다. 김책공대에서 배운 지식을 쓸 곳은 없었다.

이렇게 살다가 어느새 결혼할 나이가 됐다. 그래도 허 씨는 명색이 탄광마을이 배출한 수재이고, 평양에서 김책공대까지 나왔던 터라 여성들에게 나름 인기가 있었다.

32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 목적이었다. 저녁에 넘어가서 친척에게 전화를 하고 새벽이 되기 전에 다시 북으로 넘어왔다.

2012년 2월 그는 두 번째로 중국으로 갔다. 당시는 김정일이 사망한지 얼마 안됐던 때라 경계가 매우 심했다. 그는 북한군 군복을 입고 길을 떠났다. 군인은 초소에서 잘 단속하지 않았기 때문이다. 강을 따라 난 도로로 한참을 가다가 어둠이 내릴 때 바로 두만강을 넘었다. 중국 부락의 아무 집이나 들어가 왕청 친척에게 전화를 좀 하고 싶다고 했다. 전화를 하고 몇 시간 정도 머물렀는데 갑자기 공안이 들이닥쳤다. 집주인이 그를 도와주는 척하고는 신고를 했던 것이다. 알고 보니 중국은 그즈음부터 탈북자를 신고하면 포상금을 주는 제도를 운영하고 있었다. 특히 북한군 국경경비대가 수시로 건너와 사람까지 죽이며 노략질을 했기 때문에, 중국에서 북한군은 최고의 기피대상이었다.

탈북자들이 북송 전 대기하는 도문변방수용소에 끌려갔는데 며칠 뒤 보위부에서 차를 갖고 건너와 그를 싣고 갔다.

그렇지만 그는 예상과는 달리 20일 정도 가수감됐다가 석방됐다. 서류를 보니 중국에 친척이 다 있는 것도 확실하니, 도움 좀 받으려 넘어갔을 뿐이라는 그의 말이 인정됐다. 탈북은 배반이지만, 그의 경우엔 일탈 정도로 간주됐다.

보위부라고 해봐야 어차피 한동네 사는 아는 사람들이었는데, 그들은 지역이 배출한 인재의 경력을 망가뜨리고 싶지 않았는지 그다지 혹독하게 대하진 않았다.

하지만 허 씨 입장에선 아무 것도 이루지 못하고 잡혀오니 화가 났다. 보위부 감방에서 그는 강 건너편에서 일하다 잡혀왔다는 사람을 알게 됐다. 그는 자기가 일했던 중국 훈춘의 한 슈퍼의 이름을 알려주면서, 다시 건너가 자기 이름을 대면 사장이 고용해 줄 것이라고 했다.

감방에서 나온 허 씨는 다시 두만강을 넘었다. 세 번째 탈북이었다. 그는 감옥 동기가 알려준 슈퍼로 갔다. 왕청에 가지 않은 이유는 친척들은 별로 도와줄 의향이 없어 보였기 때문이다.
슈퍼는 중국과 나진선봉을 연결하는 도로 옆에 있었는데, 두만강 건너 허 씨네 동네가 약 10㎞ 밖에 빤히 바로 보였다. 허 씨가 내륙 깊이 들어가지 않은 이유는 딸을 데려오고 싶어서였다. 고향 가까이 있어야 집과 연락이 수월하다고 판단했다. 허 씨가 일하는 슈퍼에는 북한을 오가는 운전기사들이 수시로 드나들었다. 이들을 통해 그는 전처에게 휴대전화를 전달할 수 있었다.

● 자수해 감옥에 가다
그렇게 1년쯤 지났는데 사고가 생겼다. 중국 휴대전화를 받은 전처가 그걸 이용해 돈 벌려고 브로커 일을 하다가 보위부에 체포된 것이다. 전 남편마저 실종되니 보위부는 그녀에게 한국행을 기도했다는 누명을 씌웠다.

그 소식이 허 씨에게도 전달됐다. 그래도 4년 간 살았던 정도 있고, 어린 딸까지 있으니 모르는 척 할 수가 없었다. 그는 다시 북에 나가기 위해 자수하기로 결심했다. 자신이 나타나야 전 남편이 한국에 갔다는 누명을 벗길 수 있을 것 같았다.

슈퍼 주인은 전과자 출신이었는데, 그가 북으로 가겠다고 하자 감옥에서 나오는 자기의 비법을 전수해주었다. 폐 주변에 황산철을 주사하면 고열이 나고, 염증이 생긴다면서, 자신은 그 방법으로 병원에 호송됐다가 도망쳤다고 했다. 단 이 방법의 단점은 한 달 안에 황산철을 세척하지 못하면 생명을 잃을 수도 있다고 했다.

허 씨는 그의 조언에 따라 폐에 황산철을 네 군데나 주입했다. 그리고 스스로 공안에 자수한 뒤 2014년 4월 13일 북송됐다. 그런데 이번은 보위부도 호락호락하지 않았다. 불행하게도 그가 자수한 뒤 공안이 그의 숙소를 뒤져 소지품을 북송할 때 함께 보냈던 것이다. 그 속에는 한국 영사관, 한국인 전도사 등의 전화번호가 적힌 수첩이 있었다. 이게 화근이 됐다. 고문이 시작됐다.

중국 사장이 알려준 방법은 확실히 효과가 있었다. 폐가 곪아가기 시작했던 것이다. 염증으로 심한 열까지 나 거동을 못할 정도가 됐다. 보위부는 감방에서 죽이긴 싫었는지 그를 가석방으로 집에 보냈다.

그때가 5월 말이었다. 중국 사장이 당부한 폐를 씻어야 하는 한 달이 지난 것이다. 집에 가서 이러저런 치료를 해봤지만 점점 악화됐다. 이렇게 지내다간 죽을 것 같았다. 그는 황산철 주사를 맞은 지 네 달이 지난 8월 23일 다시 중국으로 넘어왔다. 이번엔 하얼빈에 들어가 식당에 취직해 치료에만 전념했다. 하지만 몸은 점점 더 쇠약해졌다.

몸을 운신하기 어려워지자 허 씨는 죽더라도 고향에 가서 죽겠다고 생각했다. 그해 11월말 그는 다시 연길에 왔다. 연길에서 혹시나 도움을 받지 않을까 싶어 처음으로 교회에 찾아갔는데, 거기서 집도 제공하고 치료도 해주었다. 12월 말 허 씨는 산소 호흡기에 의지하는 신세가 됐다. 쇼크도 수시로 왔다. 교회에선 한국에 가서 치료를 받아야 살 수 있다며 한국행선을 주선해주었다.


교회 목회자가 될 꿈을 꾸던 당시의 허 씨. 2017년 뉴욕에서 찍은 사진이다.


● 최초로 남북 측량기사 자격 모두 획득
2015년 2월 그는 여러 탈북민들과 함께 한국으로 떠났다. 동남아 정글에서 일행을 따라갈 수 없을 때 친한 동생이 그를 업고 산을 넘었다. 우여곡절 끝에 한국에 도착했지만 다음날 적십자병원에 실려갔다. 이곳에서 4개월 동안 치료를 받다보니 하나원도 나오지 못했다.

허 씨는 인천에 거주지가 배정됐다. 건강이 너무 악화돼 일을 할 수 없는 상태라 2016년말까지 병원에 다니며 통원치료를 받았다. 하나원을 나온 다른 사람들이 하나둘 취직해 일자리를 얻는 것을 보고 조급한 마음에 노량진에서 공무원 시험을 준비하기도 했지만, 건강이 악화돼 다시 병원에 실려 가기도 했다. 한국의 의술은 역시 대단했다. 2년이라는 오랜 시간이 걸리긴 했지만, 끝내 허 씨를 완치시켰다.

치료 받는 기간에 허 씨는 목회자의 길을 걸어볼까 싶어 1년 남짓 성경공부도 했고, 화장품 회사에 취직해 일도 해봤지만 적성에 맞지 않았다.

많은 고민 끝에 북에서 배운 측량기사 일을 다시 해보려 알아보니, 남쪽도 측량기사는 일도 힘들고 보상도 적었다. 그렇지만 적성에 맞지 않는 곳에서 시간을 낭비하는 것보다는 그래도 자신있는 분야를 파보자는 생각에 2018년 한 엔지니어링 회사에 취직했다. 취직은 했어도 아무런 건설기술인 등급도 받지 못했기 때문에 회사에서 가장 많은 나이임에도 허드레 일만 해야 했다. 연봉은 2300만 원이었다. 그는 앞으로 살아가려면 자격증이 필수라는 것을 깨달았다. 어떤 자격을 갖고 있고, 어떤 경력을 쌓는가에 따라 연봉도 결정됐다.

허 씨는 회사에 다니며 1년 동안은 토목계측에 대한 용어부터 수첩에 적어 기본적인 지식을 배웠다. 그리고 이듬해 대구과학대에 입학했다. 일도 하면서 대학 공부까지 하려니 야간반을 다닐 수밖에 없었는데, 전국의 측량 관련 야간반 중에 대구가 그나마 가까웠다.

가깝다고 해도 당시 일을 하던 포항의 건설현장에서 150㎞나 떨어져 있었다. 일주일에 서너번씩 왕복 300㎞를 달려 수업을 듣고 오는 일은 결코 쉽지 않았다.

돌아오는 길에 너무 피곤해 졸음 쉼터에서 쪽잠을 자기 일쑤였다. 잠깐 눈을 붙인다는 것이 해가 중천에 걸릴 때까지 잔적도 있다.

허 씨는 대학 공부와 병행해 자격증 시험도 준비했다. 3년 동안 주경야독의 삶을 끈질기게 이어나간 끝에 그는 2022년 대학을 우수한 성적으로 졸업하고, 동시에 측량 및 지형공간정보기사와 토목기사 자격증을 받을 수 있었다. 허 씨는 남과 북에서 측량기사 자격을 각각 받은 최초의 사례다.

이러한 자격증과 경력을 인정받아 현재 허 씨는 건설기술인협회에서 고급기술인으로 인정받고 있으며, 300억 원 규모 이상의 토목공사를 책임지고 할 수 있다.


2019년 여수에서 전력 케이블용 해저터널 공사장에서 작업하는 모습. 뒷모습이 보이는 사람이 허 씨이다.


● "배워야 살 수 있다"
허 씨는 김책공대에서 무려 8년이나 공부를 했지만 여기선 모든 것을 다시 배워야 했다고 말했다.

"남쪽에 오니 용어는 물론, 장비나 자재 등 모든 것이 북한과 달랐습니다. 비유하면 북에서 통나무로 집을 짓는 법을 배웠는데, 남쪽에 오니 시멘트로 아파트를 짓는 격이죠. 그럼 집 짓는 법을 처음부터 다시 배워야하죠. 물론 북한 김책공대 과정이 전혀 의미 없는 것은 아닙니다. 북한에선 가장 기본적인 것, 즉 공부하는 방법을 배운 것 같습니다."

그는 남북은 통합성에서도 차이가 크다고 했다.

"여기는 한 가지를 하려면 열 가지를 알아야 합니다. 많은 것들이 서로 연결돼 있어 전체적으로 진행되죠. 반면 북한은 직무가 매우 세분화돼 있고, 맡겨진 것만 하면 됐어요."

자격증과 경력을 쌓고 나니 연봉도 빠르게 올라갔다. 김책공대까지 입학했던 북한 시골 수재가 다시 자신의 두뇌와 재능을 발휘하는 데는 오랜 시간이 걸리지 않았다. 2300만 원에서 시작했던 연봉은 6년 만에 3배 이상 높아졌다.

그는 여기에 만족하지 않았다. 그는 올해 3월 다시 대구대 공간정보전문기술 석사과정에 입학했다. 토질 및 기초기술사 자격을 취득하기 위해서다. 다른 자격증과는 달리 이 자격은 받기가 매우 어렵다. 허 씨의 계획은 향후 5년 안에 석사와 함께 기술사 자격을 획득하는 것이다.

그렇다고 공부만 할 수는 없는 일. 요즘 허 씨는 마산만에서 스팀배관 부설을 위한 해저터널을 뚫는 작업을 하고 있다. 그의 직책은 공사차장. 터널 시공 품질을 총괄하는 중요한 자리다.

건설 현장은 매우 거칠고 다툼도 많다. 외국인 근로자들도 많아 의사소통의 문제도 많다. 많은 어려움이 있지만 허 씨는 포기하지 않고 한 걸음씩 나가고 있다.

"거친 현장이라서 그런지 북에서 왔다고 우습게 보는 사람들이 많아요. 아직도 저는 '나는 여전히 이방인이구나'라는 생각을 하고 있습니다. 그렇지만 실력은 남에게 뒤진다고 생각하지 않습니다. 여러 곳에서 이직 제안도 많이 받습니다."

2023년 남북하나재단이 주관한 정착사례 발표대회에서 그는 최우수상인 통일부 장관상을 받았다. 하지만 허 씨는 지금은 정착의 첫 발자국을 뗀 것에 불과하다고 했다.

그는 자신의 이름으로 회사를 만들어 키우는 것을 다음 목표로 정했다. 그 목표가 달성되면 또 할 일이 있다. 나아가 통일이 되면 할 계획도 미리 생각해두었다.

"저는 북에 돌아가 여기서 배운 기술을 북에 전수할 겁니다. 한국은 토목 공사를 할 때 측량, 시공, 품질까지 다 알아야 합니다. 통일 되면 교통인프라를 다시 정리해야 할 것인데, 북한에서 대학을 다녔던 동창들은 통합성에 있어 크게 떨어집니다. 이런 것을 가르쳐야죠. 그리고 남북에서 모두 측량기사 자격을 받은 유일한 사람이니 다른 꿈도 있습니다. 남북공간정보 통합지도체계를 만드는 것도 중요한 목표입니다."

꿈을 말할 때 그는 가장 행복한 표정이었다."""

In [16]:
chunks = text_splitter.split_text(script)
len(chunks)

29

In [17]:
chunks[0]

"허수현은 한반도 최북단 탄광마을이 낳은 수재였다. 그는 고등학교 졸업 직전 북한 전체에서 700명에게만 수여하는 '7.15 최우등상'을 수상했다. 7.15최우등상은 김정일이 평양 남산고급중학교를 졸업한 날인 1960년 7월 15일을 기념해 1987년에 만들어진 상이다. 지금은 이 상이 특권층 자식들을 대학에 보내기 위한 발판이 돼 각종 비리와 뇌물로 얼룩져 있지만, 상이 제정된 초기 몇 년은 정말 공부 잘하는 사람에게만 수여됐다. 상을 받게 되면 곧바로 중앙급 대학에 진학할 수 있었다. 북한에선 고등중학교 졸업생 중 20% 미만이 대학이나 전문학교에 갈 수 있다는 것을 감안하면 엄청난 특혜였다. 허 씨도 김책공업종합대학(김책공대)에 입학해 8년이나 공부했다. 그리고 그때 배운 지식을 활용해 지금은 한반도 최남단인 경남 마산의 해저터널 공사장에서 시공품질을 관리하는 공사차장으로 일하고 있다. 김책공대 졸업생이 네 번의 탈북을 반복한 뒤 건강이 악화돼 남의 등에 업혀 동남아 정글을 넘어 한국까지 오게 되고, 이후 남과 북에서 동시에 측량기사 자격을 받은 최초의 기술자가 돼 해저터널 공사장에서 일하게 되기까지 삶의 과정은 결코 순탄하지 않았다. 2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨."

In [18]:
chunks[1]

'● 신분 상승의 꿈\n그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다.'

In [19]:
chunks[2]

'세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다. 온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다. 탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다. 부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다. 아버지가 사망하자 허 씨는 1년 정도 방황했다. 학교에도 나가지 않았다.'

# Vector DB
* 텍스트를 벡터로 관리하는 데이터베이스 => 유사도를 비교할 수 있다.
* 온라인은 거의 없다 (MySQL, MongoDB 가 아닌 SQLite 같은 내장 DB)

## ChromaDB

In [21]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=0)
chunks = text_splitter.create_documents([script])

len(chunks)

22

In [22]:
# text (chunk) 가 입력되면 임베딩을 통해 벡터화
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(chunks, OpenAIEmbeddings())
chroma_db

### 유사도 검색 (Similarity Search)
* 쿼리에 대한 유사도가 높은 문서를 검색해서 보여준다

In [23]:
similar_docs = chroma_db.similarity_search("주인공은 어디서 태어났는가?")
print(similar_docs[0].page_content)

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


### VectorStoreRetriver
`as_retreiver()` 메소드를 이용하면 말 그대로 "검색기"를 만들 수 있습니다. 유사도 검색과 마찬가지로 쿼리에 대한 유사도가 가장 높은 문서를 답변해 주지만, 검색에 관련된 여러 옵션을 부여하는 것이 가능하며, 추후 RAG에도 사용이 가능합니다.

In [24]:
retriver = chroma_db.as_retriever()

relevant_docs = retriver.get_relevant_documents("주인공은 어디서 태어났는가?")

/var/folders/h0/f86w2r851zn7yj1w80yw932h0000gn/T/ipykernel_27732/2304472909.py:3: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriver.get_relevant_documents("주인공은 어디서 태어났는가?")


In [25]:
print(f"문서의 갯수: {len(relevant_docs)}")
print("=" * 40 + "검색 결과" + "=" * 40)
print(relevant_docs[0].page_content)

문서의 갯수: 4
========================================검색 결과========================================
2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


In [26]:
# 상위 유사도 k 개 출력
retriver = chroma_db.as_retriever(search_kwargs={"k": 2})

relevant_docs = retriver.get_relevant_documents("주인공은 어디서 태어났는가?")

print(f"문서의 개수 : {len(relevant_docs)}")
print("=" * 40 + "검색 결과" + "=" * 40)
print(relevant_docs[0].page_content)

문서의 개수 : 2
========================================검색 결과========================================
2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


**search_type**
- 검색 알고리즘을 지정할 수 있습니다.

|타입|설명|
|:---|:---|
|similarity|기본값으로서 유사도만을 사용합니다.|
|mmr|쿼리와 관련된 항목을 검색하면서 동시에 내용의 중복을 최소화 합니다.|
|similarity_score_threshold|score_threshold 기준을 충족하는 유사도 문서가 반환됩니다.|

In [27]:
# retriever 생성. 일반 similarity 사용
retriever = chroma_db.as_retriever(search_kwargs={"k": 2})

# 유사도가 가장 높은 1개 문서를 검색
relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")

print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 40)

문서의 개수: 2
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.
32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 

In [28]:
# MMR 검색
# 유사한 문서를 찾되, 다양한 특성들을 반영하기 위해 중복된 특성을 띄지 않는 문서를 우선적으로 탐색

retriver = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 2})

relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")

print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 40)

문서의 개수: 2
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.
32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 

In [29]:
# 유사도 수치를 제한하여 검색하는 기법입니다.
#  아래 예시에서는 k:3으로 설정하여 3개의 문서를 찾기로 했지만 score_threshold: 0.735를 넘는 문서가 2개밖에 없기 때문에 2개의 결과만 확인됩니다.

retriever = chroma_db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.735},
)
relevant_docs = retriever.get_relevant_documents("주인공은 어디서 태어났는가?")
print(f"문서의 개수: {len(relevant_docs)}")
print("[검색 결과]\n")
for i in range(len(relevant_docs)):
    print(relevant_docs[i].page_content)
    print("===" * 20)

문서의 개수: 2
[검색 결과]

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.
32살 때인 2006년 그는 10살 어린 여성과 결혼했다. 아내는 탄광 선전대 가수로 나름 인기가 좋았다. 이성적인 지식인과 감성적인 예술인의 조합이었다.

그렇지만 매력과 결혼생활은 별개였다. 결혼한 직후부터 둘은 다투는 일이 많았다. 서로가 서로를 견디기 어려워했다. 4년간의 결혼생활 끝에 둘은 이혼하기로 합의했다. 어린 딸은 아내가 키우기로 했다. 이때부터 그는 중국으로 눈을 돌렸다.


2022년 인천지하철 공사 현장에서 일하고 있는 허 씨.


● 두만강을 넘나들다
강 건너 연변 왕청에는 아버지의 삼촌과 사촌 등 친척들이 살고 있었다. 국경 근처라 많은 사람들이 중국을 드나들 때도 허 씨는 명색이 김책공대 졸업생인데 조국을 배반하면 안 된다는 마음이 컸다. 하지만 이혼 후 세상이 달리 보이기 시작했다. 이혼까지 한 마당에 눈치 볼 일도, 무서울 일도 없었다.

2010년 겨울 그는 국경경비대에 돈을 쥐어주고 두만강을 넘었다. 탈북한 것은 아니고, 친척에게 도움만 받자는 

## FAISS

* Meta 에서 만든 Semantic Search 를 도와주는 라이브러리
* cpu, gpu 모두 지원

In [31]:
type(chunks)

list

In [32]:
from langchain_community.vectorstores import FAISS

faiss_db = FAISS.from_documents(chunks, OpenAIEmbeddings())

In [33]:
similar_docs = faiss_db.similarity_search("주인공은 어디서 태어났는가?")

print(f"문서의 개수 : {len(similar_docs)}")
print("=====검색 결과=====")
print(similar_docs[0].page_content)

문서의 개수 : 4
=====검색 결과=====
2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


In [34]:
# 데이터베이스를 검색기로 사용하기.
retriever = faiss_db.as_retriever()

In [35]:
docs = retriever.invoke("주인공은 어디서 태어났는가?")

print(docs[0].page_content)

2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.


● 신분 상승의 꿈
그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.

온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.

탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.
부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.


In [36]:
# similarity_search_with_score : 쿼리 문서 간의 거리 점수로 리턴. 낮을 수록 유사한 문서

docs_and_scores = faiss_db.similarity_search_with_score("주인공은 어디서 태어났는가?")
docs_and_scores[0]

(Document(metadata={}, page_content='2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.\n\n\n● 신분 상승의 꿈\n그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.\n\n온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.\n\n탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.\n부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.'),
 0.33323234)

In [37]:
# similarity_search_by_vector : 임베딩 벡터와 유사한 문서를 검색
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

# 질문(질의)을 임베딩 벡터로 변환
query = "주인공은 어디서 태어났는가?"
embedding_vector = embeddings.embed_query(query)

# 임베딩 벡터를 사용해서 유사도 검색을 수행하고, 문서와의 점수를 반환
docs_and_scores = faiss_db.similarity_search_by_vector(embedding_vector)
docs_and_scores[0]

Document(metadata={}, page_content='2021년 강원도 동해시 한 아파트 공사장에서 시공 측량 작업을 진행하고 있는 허 씨.\n\n\n● 신분 상승의 꿈\n그가 태어난 한반도 최북단 온성군은 오랜 역사를 가지고 있다. 세종 22년인 1440년에 김종서 장군이 이곳을 평정한 뒤 군을 설치하고 온성이라 부르기 시작했다.\n\n온성에는 평안남도 안주 탄전에 이은 북한 최대의 갈탄 탄전이 있다. 1980년대까지만 해도 온성에는 연간 50만 톤 이상의 갈탄을 생산하는 탄광이 여러 개 있었다. 허 씨는 이런 대형 탄광 중 하나인 주원 탄광마을에서 1974년에 태어났다.\n\n탄광에서 일하는 사람들은 대개 출신성분이 나빴다. 허 씨의 부친도 마찬가지였다. 부친은 1960년대 후반까지 중국에서 살다가 문화대혁명 등의 격변기를 거치며 북한으로 넘어왔다.\n부친은 늘 허 씨에게 "너는 공부를 잘해 꼭 신분 상승을 해야 한다"고 말했다. 허 씨가 13세 때 부친은 갱이 붕괴돼 사망했다. 탄광마을에선 자주 일어나는 일이었다.')

## Emsemble Retriever

Ensemble Retriever는 여러 retriever를 입력으로 받아 get_relavant_documents() 메소드의 결과를 앙상블하고, Reciprocal Rank Fusion 알고리즘을 기반으로 결과를 재 순위화 합니다.

서로 다른 알고리즘들의 장점을 활용하므로서 EnsembleRetriever는 단일 알고리즘보다 더 나은 성능을 달성할 수 있습니다.

가장 일반적인 패턴은 sparse retriever와 dense retriever를 결합하는 것인데, 이는 두 retriever의 장점이 상호 보완적이기 때문입니다. 이를 하이브리드 검색(Hybrid Search)라고도 합니다.

Sparse Retriever는 키워드를 기반으로 관련 문서를 찾는 데 효과적이며, Dense Retriever는 의미적 유사성을 기반으로 관련 문서를 찾는데 효과적입니다.


In [38]:
%pip install rank_bm25

Note: you may need to restart the kernel to use updated packages.


In [39]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings

In [40]:
# 비타민 별 섭취할 수 있는 음식 정보 (sparse; 키워드 기반)
doc_list_1 = [
    "비타민A : 당근, 시금치, 감자 등의 주황색과 녹색 채소에서 섭취할 수 있습니다.",
    "비타민B : 전곡물, 콩, 견과류, 육류 등 다양한 식품에서 찾을 수 있습니다.",
    "비타민C : 오렌지, 키위, 딸기, 브로콜리, 피망 등의 과일과 채소에 많이 들어 있습니다.",
    "비타민D : 연어, 참치, 버섯, 우유, 계란 노른자 등에 함유되어 있습니다.",
    "비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.",
]

# 비타민 별 효능 정보 (dense; 의미 기반)
doc_list_2 = [
    "비타민A : 시력과 피부 건강을 지원합니다.",
    "비타민B : 에너지 대사와 신경계 기능을 돕습니다.",
    "비타민C : 면역 체계를 강화하고 콜라겐 생성을 촉진합니다.",
    "비타민D : 뼈 건강과 면역 체계를 지원합니다.",
    "비타민E : 항산화 작용을 통해 세포를 보호합니다.",
]

In [41]:
bm25_retriever = BM25Retriever.from_texts(
    doc_list_1, metadatas=[{"source": 1}] * len(doc_list_1)
)

bm25_retriever.k = 1  # 검색 결과 갯수를 1개로 제한

In [42]:
embeddings = OpenAIEmbeddings()

faiss_db = FAISS.from_texts(
    doc_list_2, embeddings, metadatas=[{"source": 2}] * len(doc_list_2)
)

faiss_retriever = faiss_db.as_retriever(search_kwargs={"k": 1})

In [43]:
ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, faiss_retriever], weights=[0.6, 0.4], search_type="mmr"
)

In [44]:
ensemble_result = ensemble_retriever.get_relevant_documents("비타민 A의 효능을 알려줘")
ensemble_result

[Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'),
 Document(metadata={'source': 2}, page_content='비타민A : 시력과 피부 건강을 지원합니다.')]

In [45]:
ensemble_result = ensemble_retriever.get_relevant_documents(
    "비타민 E는 어떤 식품을 통해 섭취하나요?"
)
ensemble_result

[Document(metadata={'source': 1}, page_content='비타민E : 해바라기씨, 아몬드, 시금치, 아보카도 등에서 섭취할 수 있습니다.'),
 Document(metadata={'source': 2}, page_content='비타민E : 항산화 작용을 통해 세포를 보호합니다.')]